In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

from gymnax.environments.classic_control.cartpole import EnvParams, CartPole
from rejax import get_algo
from rejax.evaluate import evaluate

import _pickle as pickle
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import yaml

In [ ]:
config_path = "./configs/custom/cartpole.yaml"

with open(config_path, "r") as f:
    config = yaml.safe_load(f.read())

In [ ]:
algo_name = "ppo"
seed_id = 42
num_seeds = 500

key = jax.random.PRNGKey(seed_id)
keys = jax.random.split(key, num_seeds)
config = config[algo_name]

In [ ]:
config

In [ ]:
env = CartPole()
config["env"] = env

gravities = jax.random.uniform(key, shape=(num_seeds,))

algo_cls = get_algo(algo_name)
algo = jax.vmap(
    lambda gravity: algo_cls.create(
        **config,
        env_params=EnvParams(
            gravity=gravity,
            max_steps_in_episode=200,
        ),
    )
)(
    gravities
)


def eval_callback(algo, ts, rng):
    act = algo.make_act(ts)
    max_steps = algo.env_params.max_steps_in_episode
    return evaluate(act, rng, env, algo.env_params, 50, max_steps)

algo = algo.replace(
    eval_callback=eval_callback
)

In [ ]:
vmap_train = jax.jit(jax.vmap(algo_cls.train, in_axes=(0, 0)))
ts, (_, returns) = vmap_train(algo, keys)
returns.block_until_ready()

print(None)

In [ ]:
pickle.dump(
    {
        "buffer_info": {k: v for k, v in ts.store_buffer.__dict__.items() if k != "data"},
        "data": {k: np.array(v) for k, v in ts.store_buffer.data._asdict().items()},
        "algorithm": {
            "algo": algo_name,
            **{k: v for k, v in config.items() if k != "env"},
        },
        "env": type(config["env"]).__name__,
        "env_params": gravities,
        "observation_space": env.observation_space(EnvParams()),
        "action_space": env.action_space(EnvParams())
    },
    open("learning_hist-cartpole-num_tasks_{}-seed_{}-{}.pkl".format(num_seeds, seed_id, algo_name), "wb"),
)

In [ ]:
assert 0

In [ ]:
regrets = (np.arange(ts.store_buffer.data.reward.shape[1])[None] + 1) * np.max(env_params, axis=-1, keepdims=True) - np.cumsum(ts.store_buffer.data.reward, axis=-1)

for regret in regrets[:5]:
    xrange = np.arange(len(regret))
    plt.plot(xrange, regret)
plt.show()


In [ ]:
for env_returns, env_param in zip(returns[:2], env_params):
    xrange = np.arange(len(env_returns)) * config["eval_freq"]
    regret = np.max(env_param, axis=-1) - env_returns
    print(regret.shape)
    mean = np.mean(regret, axis=-1)
    std = np.std(regret, axis=-1) / np.sqrt(regret.shape[-1])
    plt.plot(xrange, mean)
    plt.fill_between(xrange, mean - std, mean + std, alpha=0.2)
plt.show()


In [ ]:
for env_returns in returns[:10]:
    xrange = np.arange(len(env_returns)) * config["eval_freq"]
    mean = np.mean(env_returns, axis=-1)
    std = np.std(env_returns, axis=-1) / np.sqrt(env_returns.shape[-1])
    plt.plot(xrange, mean)
    plt.fill_between(xrange, mean - std, mean + std, alpha=0.2)
plt.show()
